# Example - Positive ZOC with a Clayton Copula

_Author:_ Karl-Ludwig Besser, Technische Universität Braunschweig

This notebook is part of the paper "On Fading Channel Dependency Structures with a Positive Zero-Outage Capacity" ([arXiv:2102.02541](https://arxiv.org/abs/2102.02541)).  
If you use any of this work, please cite the above paper.

> If you are not familiar with Jupyter notebooks: The easiest way to use this notebook interactively, is to hit `Kernel --> Restart & Run All` in the menu. This will execute all cells and enable the interactive elements of the plots.  
> Alternatively, you can execute the cells one by one using Shift+Return

In [ ]:
import numpy as np
from scipy import stats
from scipy import optimize
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
from copulas import clayton_samples

In [ ]:
def plot_clayton_samples(num_samples=1000):
    fig, axs = plt.subplots()
    ux = np.random.rand(num_samples)
    
    def update(theta=0., snr_x=0., snr_y=0.):
        snr_x = 10**(snr_x/10.)
        snr_y = 10**(snr_y/10.)
        rv_x = stats.expon(scale=snr_x)
        rv_y = stats.expon(scale=snr_y)
        x_samples, y_samples = clayton_samples(num_samples, theta=theta, u=ux, rv_x=rv_x, rv_y=rv_y)
        axs.clear()
        axs.scatter(x_samples, y_samples, fc='b', ec='k', s=10)
        axs.set_xlabel("Channel Gains $X_1$")
        axs.set_ylabel("Channel Gains $X_2$")
        axs.set_xlim([0, 6])
        axs.set_ylim([0, 6])
        if snr_x == snr_y and theta != 0:
            s_sc = rv_y.ppf(2**(1/theta))
            axs.plot([s_sc, s_sc, 0], [0, s_sc, s_sc], 'r', label="SC")
            s_mrc = 2*s_sc
            axs.plot([0, s_mrc], [s_mrc, 0], 'g', label="MRC")
            axs.legend()
    interact(update, theta=(-1., 0., .05), snr_x=(-5, 5, .5), snr_y=(-5, 5, .5))

In the following plot, you can see joint Rayleigh fading gain samples, i.e,. $X_1\sim\exp(1/\rho_x)$ and $X_2\sim\exp(1/\rho_y)$.
The joint distribution follows an underlying Clayton copula with parameter $\theta$.
For $\theta=0$, one obtains independent realizations of $X_1$ and $X_2$, while $\theta<0$ indicates negative dependence.

In [ ]:
plot_clayton_samples()

In [ ]:
def export_simulation_results(snr_db, num_samples, theta):
    snr_x = 10**(snr_db/10.)
    snr_y = 10**(snr_db/10.)
    rv_x = stats.expon(scale=snr_x)
    rv_y = stats.expon(scale=snr_y)
    ux = np.random.rand(num_samples)
    x_samples, y_samples = clayton_samples(num_samples, theta=theta, u=ux, rv_x=rv_x, rv_y=rv_y)
    import pandas as pd
    filename = "samples_rayleigh_clayton_theta{:.2f}_snr{:.2f}.dat".format(theta, snr_db)
    df = pd.DataFrame.from_dict({"x": x_samples, "y": y_samples})
    df.to_csv(filename, index=False, sep='\t')

In [ ]:
export_simulation_results(0., 2000, 0.)
export_simulation_results(0., 2000, -0.75)
export_simulation_results(0., 2000, -0.85)